In [ ]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import requests
import SimpleITK as sitk
import pathlib

In [ ]:
import os
import pandas as pd
import numpy as np
import anndata
import time
import matplotlib.pyplot as plt
import json
import requests
import pickle
import gzip as gz

import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src/')
from CellType_PSY import *

In [ ]:
## Load CCF Annotations 

In [ ]:
CCF2020 = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/abc_download_root/metadata/Allen-CCF-2020/20230630/"

In [ ]:
file = "parcellation_to_parcellation_term_membership_acronym.csv"
parcellation_annotation = pd.read_csv("{}/views/{}".format(CCF2020, file))
parcellation_annotation.set_index('parcellation_index',inplace=True)
parcellation_annotation.columns = ['parcellation_%s'% x for x in  parcellation_annotation.columns]

In [ ]:
file = "parcellation_to_parcellation_term_membership_color.csv"
parcellation_color = pd.read_csv("{}/views/{}".format(CCF2020, file))
parcellation_color.set_index('parcellation_index',inplace=True)
parcellation_color.columns = ['parcellation_%s'% x for x in  parcellation_color.columns]

In [ ]:
CCF_V3_ontology = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/CCF_V3_ISH_MERFISH.csv", index_col=0)

In [ ]:
ASD_STR_ISH_Bias = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", 
                               index_col=0)
Structures_ISH = []
for STR in ASD_STR_ISH_Bias.index.values:
    New_Name = STR.replace("_", " ")
    Structures_ISH.append(New_Name)

In [ ]:
def Add_ISH_STR(cell_meta, CCF_V3_ontology, Structures_ISH):
    for i, row in cell_meta.iterrows():
        _str = row["parcellation_structure"]
        _substr = row["parcellation_substructure"]
        if _str in CCF_V3_ontology.index.values:
            name_str = CCF_V3_ontology.loc[_str, "CleanName"]
        else:
            name_str = "None"
        if _substr in CCF_V3_ontology.index.values:
            name_substr = CCF_V3_ontology.loc[_substr, "CleanName"]
        else:
            name_substr = "None"
        if name_str in Structures_ISH: # should capture most of them
            ISH_STR = name_str
        elif name_substr in Structures_ISH: # should capture AM IC MG MDRN
            ISH_STR = name_substr
        elif _str in ["VISa", "VISrl"]:
            ISH_STR = "Posterior parietal association areas"
        elif name_str == "Subiculum":
            ISH_STR = "Subiculum"
        else:
            ISH_STR = "Not in Connectome"
        cell_meta.loc[i, "ISH_STR"] = ISH_STR
        cell_meta.loc[i, "ISH_STR2"] = "_".join(ISH_STR.split())
    return cell_meta

### Test

In [ ]:
MERFISH_Dir = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/MERFISH_ZhuangLab/ABCA-4/"
outname = "cell_metadata_with_cluster_str_annotation.csv"

In [ ]:
cell_meta = pd.read_csv("{}/cell_metadata_with_cluster_annotation.csv".format(MERFISH_Dir), index_col=0)
cell_meta.rename(columns={'x':'x_section','y':'y_section','z':'z_section'},inplace=True)
cell_coords = pd.read_csv("{}/ccf_coordinates.csv".format(MERFISH_Dir), index_col=0)
cell_coords.rename(columns={'x':'x_ccf','y':'y_ccf','z':'z_ccf'},inplace=True)
cell_meta = cell_meta.join(cell_coords,how='inner')
cell_meta = cell_meta.join(parcellation_annotation,on='parcellation_index')
cell_meta = cell_meta.join(parcellation_color,on='parcellation_index')
cell_meta = Add_ISH_STR(cell_meta, CCF_V3_ontology, Structures_ISH)
cell_meta.to_csv("{}/{}".format(MERFISH_Dir, outname))

In [ ]:
MERFISH_Dir = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/MERFISH_ZhuangLab/ABCA-1/"
outname = "cell_metadata_with_cluster_str_annotation.csv"

cell_meta = pd.read_csv("{}/cell_metadata_with_cluster_annotation.csv".format(MERFISH_Dir), index_col=0)
cell_meta.rename(columns={'x':'x_section','y':'y_section','z':'z_section'},inplace=True)
cell_coords = pd.read_csv("{}/ccf_coordinates.csv".format(MERFISH_Dir), index_col=0)
cell_coords.rename(columns={'x':'x_ccf','y':'y_ccf','z':'z_ccf'},inplace=True)
cell_meta = cell_meta.join(cell_coords,how='inner')
cell_meta = cell_meta.join(parcellation_annotation,on='parcellation_index')
cell_meta = cell_meta.join(parcellation_color,on='parcellation_index')
cell_meta = Add_ISH_STR(cell_meta, CCF_V3_ontology, Structures_ISH)
cell_meta.to_csv("{}/{}".format(MERFISH_Dir, outname))

In [ ]:
MERFISH_Dir = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/MERFISH_ZhuangLab/ABCA-2/"
outname = "cell_metadata_with_cluster_str_annotation.csv"

cell_meta = pd.read_csv("{}/cell_metadata_with_cluster_annotation.csv".format(MERFISH_Dir), index_col=0)
cell_meta.rename(columns={'x':'x_section','y':'y_section','z':'z_section'},inplace=True)
cell_coords = pd.read_csv("{}/ccf_coordinates.csv".format(MERFISH_Dir), index_col=0)
cell_coords.rename(columns={'x':'x_ccf','y':'y_ccf','z':'z_ccf'},inplace=True)
cell_meta = cell_meta.join(cell_coords,how='inner')
cell_meta = cell_meta.join(parcellation_annotation,on='parcellation_index')
cell_meta = cell_meta.join(parcellation_color,on='parcellation_index')
cell_meta = Add_ISH_STR(cell_meta, CCF_V3_ontology, Structures_ISH)
cell_meta.to_csv("{}/{}".format(MERFISH_Dir, outname))

In [ ]:
MERFISH_Dir = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/MERFISH_ZhuangLab/ABCA-3/"
outname = "cell_metadata_with_cluster_str_annotation.csv"

cell_meta = pd.read_csv("{}/cell_metadata_with_cluster_annotation.csv".format(MERFISH_Dir), index_col=0)
cell_meta.rename(columns={'x':'x_section','y':'y_section','z':'z_section'},inplace=True)
cell_coords = pd.read_csv("{}/ccf_coordinates.csv".format(MERFISH_Dir), index_col=0)
cell_coords.rename(columns={'x':'x_ccf','y':'y_ccf','z':'z_ccf'},inplace=True)
cell_meta = cell_meta.join(cell_coords,how='inner')
cell_meta = cell_meta.join(parcellation_annotation,on='parcellation_index')
cell_meta = cell_meta.join(parcellation_color,on='parcellation_index')
cell_meta = Add_ISH_STR(cell_meta, CCF_V3_ontology, Structures_ISH)
cell_meta.to_csv("{}/{}".format(MERFISH_Dir, outname))

In [ ]:
def GetBiasesForSTR(STR, MERFISH_STRAnn, ASD_CT_Bias_Cluster, CT_ID_SET, Non_Neuron_Class):
    STR_MERFISH_DF = MERFISH_STRAnn[MERFISH_STRAnn["ISH_STR"]==STR]
    total_bias = 0
    total_bias_neuro = 0
    N_cell = 0
    N_neuro = 0
    for i, row in STR_MERFISH_DF.iterrows():
        cluster_id = row["cluster"]
        if cluster_id not in CT_ID_SET:
            continue
        class_id = row["class"]
        asd_cluster_bias = ASD_CT_Bias_Cluster.loc[cluster_id, "EFFECT"]
        if asd_cluster_bias == asd_cluster_bias:
            total_bias += asd_cluster_bias
            N_cell += 1
            if class_id not in Non_Neuron_Class:
                total_bias_neuro += asd_cluster_bias
                N_neuro += 1
    if N_cell > 0:
        bias_per_cell = total_bias/N_cell
    else:
        bias_per_cell = 0
    if N_neuro > 0:
        bias_per_neuro = total_bias_neuro/N_neuro
    else:
        bias_per_neuro = 0
    Vol = CCF_V3_STR.loc[STR, "total_voxel_counts (10 um)"]
    bias_per_Vol = total_bias/Vol * 100
    return bias_per_cell, bias_per_neuro, bias_per_Vol
#STRs = list(set(MERFISH_STRAnn["ISH_STR"]))
#STRs.remove("Not in Connectome")
#CT_ID_SET = set(ASD_ClusterV3_Bias.index.values)
Non_Neuron_Class = ["30 Astro-Epen","31 OPC-Oligo","32 OEC","33 Vascular","34 Immune"]
CCF_V3_STR = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/CCF_V3_ISH_MERFISH.csv",
                        index_col="CleanName")

In [ ]:
MERFISH_Dir1 = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/MERFISH_ZhuangLab/ABCA-1/"
MERFISH_Dir2 = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/MERFISH_ZhuangLab/ABCA-2/"
MERFISH_Dir3 = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/MERFISH_ZhuangLab/ABCA-3/"
MERFISH_Dir4 = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/MERFISH_ZhuangLab/ABCA-4/"
MERFISH1 = pd.read_csv("{}/{}".format(MERFISH_Dir1, outname))
MERFISH2 = pd.read_csv("{}/{}".format(MERFISH_Dir2, outname))
MERFISH3 = pd.read_csv("{}/{}".format(MERFISH_Dir3, outname))
MERFISH4 = pd.read_csv("{}/{}".format(MERFISH_Dir4, outname))

In [ ]:
print(MERFISH1.shape)
print(MERFISH2.shape)
print(MERFISH3.shape)
print(MERFISH4.shape)

In [ ]:
MERFISH_Combine = pd.concat([MERFISH1, MERFISH2, MERFISH3, MERFISH4])

In [ ]:
MERFISH_Combine.to_csv("dat/MERFISH_Zhuang/MERFISH_Zhuang_1-4_Combine.STR.Annot.csv")

In [ ]:
ASD_ClusterV3_Bias = pd.read_csv("dat/Bias/ASD.ClusterV3.top60.DN.Z2.csv", index_col=0)
STRs = list(set(MERFISH1["ISH_STR"]))
STRs.remove("Not in Connectome")
CT_ID_SET = set(ASD_ClusterV3_Bias.index.values)

#### test with cell meta 4

In [ ]:
bias_dat_percell = []
bias_dat_perneuro = []
bias_dat_pervol = []
for STR in STRs:
    bias_percell, bias_per_neuro, bias_per_vol = GetBiasesForSTR(STR, MERFISH2, ASD_ClusterV3_Bias,
                                                                 CT_ID_SET, Non_Neuron_Class)
    bias_dat_percell.append(bias_percell)
    bias_dat_perneuro.append(bias_per_neuro)
    bias_dat_pervol.append(bias_per_vol)
    print(STR, bias_percell, bias_per_neuro, bias_per_vol)

In [ ]:
AggV2_ASD_DF_MERFISH2 = pd.DataFrame(data={"STR2":STRs, "EFFECT_perCell":bias_dat_percell, "EFFECT_perNeuro":bias_dat_perneuro,
                                 "EFFECT_perVol":bias_dat_pervol, })
AggV2_ASD_DF_MERFISH2 = AggV2_ASD_DF_MERFISH2.set_index("STR2")

In [ ]:
ASD_ISH = pd.read_csv("dat/Bias/ASD.ISH.bias.csv", index_col=0)
ASD_MERFISH_adj = pd.read_csv("dat/Bias/ASD.MERFISH.Agg.adj.bias.csv", index_col=0)

In [ ]:
for i, row in AggV2_ASD_DF.iterrows():
    
    STR = "_".join(i.split())
    if STR not in ASD_MERFISH_adj.index.values:
        AggV2_ASD_DF.loc[i, "STR"] = ""
        AggV2_ASD_DF.loc[i, "REGION"] = ""
    else:
        REG = ASD_MERFISH_adj.loc[STR, "REGION"]
        AggV2_ASD_DF.loc[i, "STR"] = STR
        AggV2_ASD_DF.loc[i, "REGION"] = REG
    
    
    AggV2_ASD_DF.loc[i, "EFFECT_ISH"] = ASD_ISH.loc[STR, "EFFECT"]
AggV2_ASD_DF = AggV2_ASD_DF.sort_values("EFFECT_perVol", ascending=False)

In [ ]:
AggV2_ASD_DF.head(50)

In [ ]:
pearsonr(AggV2_ASD_DF["EFFECT_perCell"].values, AggV2_ASD_DF["EFFECT_ISH"].values)

In [ ]:
pearsonr(AggV2_ASD_DF["EFFECT_perNeuro"].values, AggV2_ASD_DF["EFFECT_ISH"].values)

In [ ]:
pearsonr(AggV2_ASD_DF["EFFECT_perVol"].values, AggV2_ASD_DF["EFFECT_ISH"].values)

In [ ]:
Bias1 = ASD_ISH
Bias2 = AggV2_ASD_DF.sort_values("EFFECT_perNeuro", ascending=False).set_index("STR")
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = ASD_ISH
Bias2 = AggV2_ASD_DF.sort_values("EFFECT_perVol", ascending=False).set_index("STR")
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = ASD_ISH
Bias2 = AggV2_ASD_DF.sort_values("EFFECT_perCell", ascending=False).set_index("STR")
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

# Test on Combine MERFISH

In [ ]:
bias_dat_percell = []
bias_dat_perneuro = []
bias_dat_pervol = []
for STR in STRs:
    bias_percell, bias_per_neuro, bias_per_vol = GetBiasesForSTR(STR, MERFISH_Combine, ASD_ClusterV3_Bias,
                                                                 CT_ID_SET, Non_Neuron_Class)
    bias_dat_percell.append(bias_percell)
    bias_dat_perneuro.append(bias_per_neuro)
    bias_dat_pervol.append(bias_per_vol)
    print(STR, bias_percell, bias_per_neuro, bias_per_vol)

In [ ]:
AggV2_ASD_DF_MERFISH_CB = pd.DataFrame(data={"STR2":STRs, "EFFECT_perCell":bias_dat_percell, "EFFECT_perNeuro":bias_dat_perneuro,
                                 "EFFECT_perVol":bias_dat_pervol, })
AggV2_ASD_DF_MERFISH_CB = AggV2_ASD_DF_MERFISH_CB.set_index("STR2")

In [ ]:
AggV2_ASD_DF_MERFISH_CB = AggV2_ASD_DF_MERFISH_CB.sort_values("EFFECT_perVol", ascending=False)

In [ ]:
for i, row in AggV2_ASD_DF_MERFISH_CB.iterrows():
    
    STR = "_".join(i.split())
    if STR not in ASD_MERFISH_adj.index.values:
        AggV2_ASD_DF_MERFISH_CB.loc[i, "STR"] = ""
        AggV2_ASD_DF_MERFISH_CB.loc[i, "REGION"] = ""
    else:
        REG = ASD_MERFISH_adj.loc[STR, "REGION"]
        AggV2_ASD_DF_MERFISH_CB.loc[i, "STR"] = STR
        AggV2_ASD_DF_MERFISH_CB.loc[i, "REGION"] = REG
    
    
    AggV2_ASD_DF_MERFISH_CB.loc[i, "EFFECT_ISH"] = ASD_ISH.loc[STR, "EFFECT"]

In [ ]:
AggV2_ASD_DF_MERFISH_CB.head(50)

In [ ]:
pearsonr(AggV2_ASD_DF_MERFISH_CB["EFFECT_perCell"].values, AggV2_ASD_DF_MERFISH_CB["EFFECT_ISH"].values)

In [ ]:
pearsonr(AggV2_ASD_DF_MERFISH_CB["EFFECT_perNeuro"].values, AggV2_ASD_DF_MERFISH_CB["EFFECT_ISH"].values)

In [ ]:
pearsonr(AggV2_ASD_DF_MERFISH_CB["EFFECT_perVol"].values, AggV2_ASD_DF_MERFISH_CB["EFFECT_ISH"].values)

In [ ]:
Bias1 = ASD_ISH
Bias2 = AggV2_ASD_DF_MERFISH_CB.sort_values("EFFECT_perCell", ascending=False).set_index("STR")
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = ASD_ISH
Bias2 = AggV2_ASD_DF_MERFISH_CB.sort_values("EFFECT_perNeuro", ascending=False).set_index("STR")
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = ASD_ISH
Bias2 = AggV2_ASD_DF_MERFISH_CB.sort_values("EFFECT_perVol", ascending=False).set_index("STR")
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
AggV2_ASD_DF_MERFISH_Allen = pd.read_csv("dat/Bias/ASD.STR.top60.MERFISH.csv", index_col=0)

In [ ]:
AggV2_ASD_DF_MERFISH_CB

In [ ]:
AggV2_ASD_DF_MERFISH_CB.head(2)

In [ ]:
AggV2_ASD_DF_MERFISH_Allen.head(2)

In [ ]:
AggV2_ASD_DF_MERFISH_XX = AggV2_ASD_DF_MERFISH_CB.join(AggV2_ASD_DF_MERFISH_Allen, on='STR2', how='inner',
                                                      lsuffix="_Zhuang", rsuffix="_Allen")

In [ ]:
AggV2_ASD_DF_MERFISH_XX.head(2)

In [ ]:
pearsonr(AggV2_ASD_DF_MERFISH_XX["EFFECT_perCell_Zhuang"].values, 
         AggV2_ASD_DF_MERFISH_XX["EFFECT_perCell_Allen"].values)

In [ ]:
pearsonr(AggV2_ASD_DF_MERFISH_XX["EFFECT_perVol_Zhuang"].values, 
         AggV2_ASD_DF_MERFISH_XX["EFFECT_perVol_Allen"].values)

In [ ]:
pearsonr(AggV2_ASD_DF_MERFISH_XX["EFFECT_perNeuro_Zhuang"].values, 
         AggV2_ASD_DF_MERFISH_XX["EFFECT_perNeuro_Allen"].values)

In [ ]:
Bias1 = AggV2_ASD_DF_MERFISH_XX.sort_values("EFFECT_perVol_Zhuang", ascending=False)
Bias2 = AggV2_ASD_DF_MERFISH_XX.sort_values("EFFECT_perVol_Allen", ascending=False)
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = AggV2_ASD_DF_MERFISH_XX.sort_values("EFFECT_perCell_Zhuang", ascending=False)
Bias2 = AggV2_ASD_DF_MERFISH_XX.sort_values("EFFECT_perCell_Allen", ascending=False)
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = AggV2_ASD_DF_MERFISH_XX.sort_values("EFFECT_perNeuro_Zhuang", ascending=False)
Bias2 = AggV2_ASD_DF_MERFISH_XX.sort_values("EFFECT_perNeuro_Allen", ascending=False)
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
AggV2_ASD_DF_MERFISH_CB

In [ ]:
STR_MERFISH_Zhuang_CM_Zmatch = pd.read_csv("dat/MERFISH_Zhuang/Z2Mat.ISH_STR.CellMean.SCMatch.Zmatch.csv", index_col=0)
STR_MERFISH_Zhuang_VM_Zmatch = pd.read_csv("dat/MERFISH_Zhuang/Z2Mat.ISH_STR.VolMean.SCMatch.Zmatch.csv", index_col=0)
head = STR_MERFISH_Z2Mat_CM.columns.values
new_head = [X.replace(" ", "_") for X in head]
STR_MERFISH_Zhuang_CM_Zmatch.columns = new_head
STR_MERFISH_Zhuang_VM_Zmatch.columns = new_head

In [ ]:
ASD_Bias_MERFISH_MIT_CM_ZM = AvgSTRZ_Weighted(STR_MERFISH_Zhuang_CM_Zmatch, ASD_GW) 
ASD_Bias_MERFISH_MIT_VM_ZM = AvgSTRZ_Weighted(STR_MERFISH_Zhuang_VM_Zmatch, ASD_GW) 

ASD_Bias_MERFISH_MIT_CM_ZM_adj = AvgSTRZ_Weighted(STR_MERFISH_Zhuang_CM_Zmatch, ASD_GW_adj) 
ASD_Bias_MERFISH_MIT_VM_ZM_adj = AvgSTRZ_Weighted(STR_MERFISH_Zhuang_VM_Zmatch, ASD_GW_adj) 

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_MIT_CM_ZM, ASD_ISH)

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_MIT_CM_ZM_adj, ASD_ISH)

In [ ]:
Bias1 = ASD_ISH
Bias2 = ASD_Bias_MERFISH_MIT_CM_ZM
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = ASD_ISH
Bias2 = ASD_Bias_MERFISH_MIT_CM_ZM_adj
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
ASD_Bias_MERFISH_Allen_CM_Zmatch = pd.read_csv("dat/Bias/MERFISH_STR_AggBias/ASD.MERFISH.Allen.CellMean.Zmatch.Z2.csv", index_col=0)
ASD_Bias_MERFISH_Allen_VM_Zmatch = pd.read_csv("dat/Bias/MERFISH_STR_AggBias/ASD.MERFISH.Allen.VolMean.Zmatch.Z2.csv", index_col=0)

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_MIT_CM_ZM_adj, ASD_Bias_MERFISH_Allen_VM_Zmatch)

In [ ]:
Bias1 = ASD_Bias_MERFISH_MIT_CM_ZM_adj
Bias2 = ASD_Bias_MERFISH_Allen_VM_Zmatch
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
STR_MERFISH_Z2Mat_CM = pd.read_csv("dat/MERFISH_Zhuang/Z2Mat.ISH_STR.CellMean.SCMatch.csv", index_col=0)
STR_MERFISH_Z2Mat_VM = pd.read_csv("dat/MERFISH_Zhuang/Z2Mat.ISH_STR.VolMean.SCMatch.csv", index_col=0)
head = STR_MERFISH_Z2Mat_CM.columns.values
new_head = [X.replace(" ", "_") for X in head]
STR_MERFISH_Z2Mat_CM.columns = new_head
STR_MERFISH_Z2Mat_VM.columns = new_head

In [ ]:
ASD_GW = Fil2Dict("../../ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.GeneWeight.csv")
GeneExpCorr = pd.read_csv("dat/ISH_MERFISH_Gene_CorssSTR_Corr.v2.csv", index_col=0)
ASD_GW_adj = {}
for k,v in ASD_GW.items():
    if k in GeneExpCorr.index.values:
        ASD_GW_adj[k] = v * (GeneExpCorr.loc[k, "V2_V3_CT_Corr"]**2)

In [ ]:
ASD_Bias_MERFISH_MIT_Z2_CM = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_CM, ASD_GW) 
ASD_Bias_MERFISH_MIT_Z2_VM = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_VM, ASD_GW) 

In [ ]:
ASD_Bias_MERFISH_MIT_Z2_CM_adj = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_CM, ASD_GW_adj) 
ASD_Bias_MERFISH_MIT_Z2_VM_adj = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_VM, ASD_GW_adj) 

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_MIT_Z2_VM, ASD_ISH)
Bias1 = ASD_ISH
Bias2 = ASD_Bias_MERFISH_MIT_Z2_VM
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_MIT_Z2_VM_adj, ASD_ISH)
Bias1 = ASD_ISH
Bias2 = ASD_Bias_MERFISH_MIT_Z2_VM_adj
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_STR_VM, ASD_ISH)

In [ ]:
Bias1 = ASD_ISH
Bias2 = ASD_Bias_MERFISH_STR_VM
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
#ASD_Bias_MERFISH_STR_VM[ASD_Bias_MERFISH_STR_VM["REGION"]=="Thalamus"]

In [ ]:
#ASD_Bias_MERFISH_STR_VM[ASD_Bias_MERFISH_STR_VM["REGION"]=="Isocortex"]

In [ ]:
ASD_Bias_MERFISH_Allen_Z2_CM = pd.read_csv("dat/Bias/MERFISH_STR_AggBias/ASD.MERFISH.Allen.CellMean.Z2.csv", index_col=0)
ASD_Bias_MERFISH_Allen_Z2_VM = pd.read_csv("dat/Bias/MERFISH_STR_AggBias/ASD.MERFISH.Allen.VolMean.Z2.csv", index_col=0)
ASD_Bias_MERFISH_Allen_Z2_CM_adj = pd.read_csv("dat/Bias/MERFISH_STR_AggBias/ASD.MERFISH.Allen.CellMean.adj.Z2.csv", index_col=0)
ASD_Bias_MERFISH_Allen_Z2_VM_adj = pd.read_csv("dat/Bias/MERFISH_STR_AggBias/ASD.MERFISH.Allen.VolMean.adj.Z2.csv", index_col=0)

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_Allen_Z2_VM, ASD_Bias_MERFISH_MIT_Z2_VM)
Bias1 = ASD_Bias_MERFISH_Allen_Z2_VM
Bias2 = ASD_Bias_MERFISH_MIT_Z2_VM
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_Allen_Z2_VM_adj, ASD_Bias_MERFISH_MIT_Z2_VM_adj)
Bias1 = ASD_Bias_MERFISH_Allen_Z2_VM_adj
Bias2 = ASD_Bias_MERFISH_MIT_Z2_VM_adj
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_STR_VM, ASD_Bias_MERFISH_Allen_VM_Zmatch)

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_MIT_Z2_VM, ASD_Bias_MERFISH_Allen_VM)
Bias1 = ASD_Bias_MERFISH_MIT_Z2_VM
Bias2 = ASD_Bias_MERFISH_Allen_VM
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = ASD_Bias_MERFISH_STR_VM
Bias2 = ASD_Bias_MERFISH_Allen_VM
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
MouseISH_Zmatch = pd.read_csv("dat/MERFISH/Z2Mat.ISH.Zmatch.csv", index_col=0)

In [ ]:
MouseISH_Z1 = pd.read_csv("../../ASD_Circuits/dat/allen-mouse-exp/AllenMouse_z1_mat.0511.csv", index_col=0)

In [ ]:
ASD_ISH_Z1 = AvgSTRZ_Weighted(MouseISH_Z1, ASD_GW) 
ASD_ISH_Z1 = FixSubiculum(ASD_ISH_Z1)

In [ ]:
BiasCorrelation(ASD_ISH_Z1, ASD_ISH)
Bias1 = ASD_ISH_Z1
Bias2 = ASD_ISH
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
BiasCorrelation(ASD_ISH_Z1, ASD_ISH_v2)
Bias1 = ASD_ISH_Z1
Bias2 = ASD_ISH_v2
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
ASD_ISH_v2 = AvgSTRZ_Weighted(MouseISH_Zmatch, ASD_GW) 
ASD_ISH_v2_adj = AvgSTRZ_Weighted(MouseISH_Zmatch, ASD_GW_adj) 

In [ ]:
def FixSubiculum(DF):
    X = DF.loc["Subiculum_dorsal_part"]
    Y = DF.loc["Subiculum_ventral_part"]
    Z = [(X[0]+Y[0])/2, "Hippocampus", 214]
    DF.loc["Subiculum"] = Z
    DF = DF.drop(["Subiculum_dorsal_part", "Subiculum_ventral_part"])
    return DF

In [ ]:
ASD_ISH_v2 = FixSubiculum(ASD_ISH_v2)
ASD_ISH_v2_adj = FixSubiculum(ASD_ISH_v2_adj)

In [ ]:
#ASD_ISH_v2.head(50)

In [ ]:
#ASD_ISH_v2[ASD_ISH_v2["REGION"]=="Thalamus"]

In [ ]:
print(RegionDistributionsList(ASD_ISH_v2.head(50).index.values))

In [ ]:
BiasCorrelation(ASD_ISH_v2, ASD_ISH)
Bias1 = ASD_ISH_v2
Bias2 = ASD_ISH
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
BiasCorrelation(ASD_ISH_v2_adj, ASD_ISH)

In [ ]:
BiasCorrelation(ASD_ISH_v2, ASD_Bias_MERFISH_STR_VM_ZM)

In [ ]:
BiasCorrelation(ASD_ISH_v2, ASD_Bias_MERFISH_Allen_VM_Zmatch)

In [ ]:
Bias1 = ASD_ISH_v2
Bias2 = ASD_Bias_MERFISH_Allen_VM_Zmatch
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

## Clean UP

In [ ]:
Merfish_Zhuang_Z2 = pd.read_csv("dat/MERFISH_Zhuang/Z2Mat.ISH_STR.VolMean.SCMatch.Zmatch.csv")


# Expression L Compare of 2 MERFISH dataset

In [ ]:
MERFISH_Allen_STR_ExpL = pd.read_csv("dat/MERFISH/STR_Vol_Mean_DF.UMI.csv", index_col=0)
MERFISH_Zhuang_STR_ExpL = pd.read_csv("dat/MERFISH_Zhuang/STR_Vol_Mean_DF.UMI.csv", index_col=0)

In [ ]:
MERFISH_Allen_STR_Z1 = pd.read_csv("dat/MERFISH/STR_Vol_Mean_Z1Mat.clip.csv", index_col=0)
MERFISH_Zhuang_STR_Z1 = pd.read_csv("dat/MERFISH_Zhuang/STR_Vol_Mean_Z1Mat.clip.csv", index_col=0)

In [ ]:
GeneExpCorr = pd.read_csv("dat/ISH_MERFISH_Gene_CorssSTR_Corr.v2.csv", index_col=0)

In [ ]:
GeneExpCorr.head(2)

In [ ]:
for i, row in GeneExpCorr.iterrows():
    row1 = MERFISH_Allen_STR_ExpL.loc[i, :].values
    row2 = MERFISH_Zhuang_STR_ExpL.loc[i, :].values
    r, p = pearsonr(row1, row2) #, nan_policy="omit")
    GeneExpCorr.loc[i, "Two_MERFISH_STR_Corr"] = r

In [ ]:
GeneExpCorr.to_csv("dat/ISH_MERFISH_Gene_CorssSTR_Corr.v3.csv")

In [ ]:
X = GeneExpCorr["V2_V3_CT_Corr"]
Y = GeneExpCorr["Two_MERFISH_STR_Corr"]
plt.scatter(X, Y, s=1)

##### Corss STRs

In [ ]:
AllenMERFISH_MKExp = pd.read_csv("dat/MERFISH_MarkerG/STR_MERFISH_MarkerG.csv", index_col=0)

In [ ]:
AllenMERFISH_MKGenes = AllenMERFISH_MKExp.index.values

In [ ]:
len(AllenMERFISH_MKGenes)

In [ ]:
Cross_STR_Rs = []
Cross_STR_Rs_MKG = []
for g, row in MERFISH_Allen_STR_ExpL.iterrows():
    row1 = row.values
    row2 = MERFISH_Zhuang_STR_ExpL.loc[g, :].values
    r, p = pearsonr(row1, row2) #, nan_policy="omit")
    Cross_STR_Rs.append(r)
    if g in AllenMERFISH_MKGenes:
        Cross_STR_Rs_MKG.append(r)

In [ ]:
plt.hist(Cross_STR_Rs, bins=20)
plt.xlabel("Correlation of ExpL cross STR for each gene")
plt.show()

In [ ]:
plt.hist(Cross_STR_Rs_MKG, bins=20)
plt.xlabel("Correlation of ExpL cross STR for each gene")
plt.show()

In [ ]:
Cross_STR_Z1_Rs = []
for g, row in MERFISH_Allen_STR_Z1.iterrows():
    row1 = row.values
    row2 = MERFISH_Zhuang_STR_Z1.loc[g, :].values
    #r, p = pearsonr(row1, row2) #, nan_policy="omit")
    r, p = spearmanr(row1, row2, nan_policy="omit")
    Cross_STR_Z1_Rs.append(r)

In [ ]:
plt.hist(Cross_STR_Z1_Rs, bins=20)
plt.xlabel("Correlation of Z1 cross STR for each gene")
plt.show()

##### Cross genes Per STR

In [ ]:
STR_Cross_Gene_Exp_Rs = []
#STR_Cross_Gene_Exp_Rs = []
for STR in MERFISH_Allen_STR_ExpL.columns.values:
    row1 = MERFISH_Allen_STR_ExpL[STR].values
    row2 = MERFISH_Zhuang_STR_ExpL[STR].values
    r, p = pearsonr(row1, row2) #, nan_policy="omit")
    STR_Cross_Gene_Exp_Rs.append(r)

In [ ]:
print(np.mean(STR_Cross_Gene_Exp_Rs))
print(np.median(STR_Cross_Gene_Exp_Rs))

In [ ]:
#plt.hist(Cross_STR_Rs, bins=20, label="ExpL Cross STR")
#plt.hist(Cross_STR_Z1_Rs, bins=20, label="Z1 Cross STR")
plt.hist(STR_Cross_Gene_Exp_Rs, bins=20, label="Exp Cross Genes")

plt.xlabel("Correlation of ExpL cross Genes for each structure")
plt.show()

In [ ]:
df = pd.DataFrame(data={"STR":MERFISH_Allen_STR_ExpL.columns.values, "GeneCorr":STR_Cross_Gene_Exp_Rs})

In [ ]:
df = df.sort_values("GeneCorr")

In [ ]:
df.head(50)

In [ ]:
df.tail(50)

## Compare Cell composition of 2 merfish

In [ ]:
MERFISH_Allen = pd.read_csv("dat/MERFISH/MERFISH.ISH_Annot.clean.csv", index_col=0)

In [ ]:
MERFISH_MIT = pd.read_csv("dat/MERFISH_Zhuang/MERFISH_Zhuang_1-4_Combine.STR.Annot.csv", index_col=0)

In [ ]:
MERFISH_Allen.columns.values

In [ ]:
ISH_STRs = set(MERFISH_Allen["ISH_STR"])

In [ ]:
def GetBiasesForSTR(STR, MERFISH_STRAnn, ASD_CT_Bias_Cluster, CT_ID_SET, Non_Neuron_Class):
    STR_MERFISH_DF = MERFISH_STRAnn[MERFISH_STRAnn["ISH_STR"]==STR]
    total_bias = 0
    total_bias_neuro = 0
    N_cell = 0
    N_neuro = 0
    for i, row in STR_MERFISH_DF.iterrows():
        cluster_id = row["cluster"]
        if cluster_id not in CT_ID_SET:
            continue
        class_id = row["class"]
        asd_cluster_bias = ASD_CT_Bias_Cluster.loc[cluster_id, "EFFECT"]
        if asd_cluster_bias == asd_cluster_bias:
            total_bias += asd_cluster_bias
            N_cell += 1
            if class_id not in Non_Neuron_Class:
                total_bias_neuro += asd_cluster_bias
                N_neuro += 1
    if N_cell > 0:
        bias_per_cell = total_bias/N_cell
    else:
        bias_per_cell = 0
    if N_neuro > 0:
        bias_per_neuro = total_bias_neuro/N_neuro
    else:
        bias_per_neuro = 0
    Vol = CCF_V3_STR.loc[STR, "total_voxel_counts (10 um)"]
    bias_per_Vol = total_bias/Vol * 100
    return bias_per_cell, bias_per_neuro, bias_per_Vol

In [ ]:
N_Cell_Allen = []
N_Cell_MIT = []
ISH_STRs_Name = []
for STR in ISH_STRs:
    if STR == 'Not in Connectome':
        continue
    STR_name = "_".join(STR.split())
    STR_MERFISH_Allen = MERFISH_Allen[MERFISH_Allen["ISH_STR"]==STR]
    STR_MERFISH_MIT = MERFISH_MIT[MERFISH_MIT["ISH_STR"]==STR]
    #print("STR:{} Allen:{} MIT:{}".format(STR, STR_MERFISH_Allen.shape[0], STR_MERFISH_MIT.shape[0]))
    ISH_STRs_Name.append(STR_name)
    N_Cell_Allen.append(STR_MERFISH_Allen.shape[0])
    N_Cell_MIT.append(STR_MERFISH_MIT.shape[0])

In [ ]:
ASD_ISH =  pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.csv", 
                               index_col=0)
ASD_ISH = FixSubiculum(ASD_ISH)
ASD_ISH["REGION"] = ASD_ISH["REGION"].replace("Amygdalar", "Amygdala")

In [ ]:
df = pd.DataFrame(data={"STR":ISH_STRs_Name, "N_Cell_Allen":N_Cell_Allen, "N_Cell_MIT":N_Cell_MIT})

In [ ]:
Regions = [ASD_ISH.loc[X, "REGION"] for X in ISH_STRs_Name]

In [ ]:
df["Region"] = Regions 

In [ ]:
df = df.set_index("STR")

In [ ]:
df.head(2)

In [ ]:
plt.scatter(np.log10(df["N_Cell_Allen"].values), np.log10(df["N_Cell_MIT"].values))

In [ ]:
STR = "Mediodorsal nucleus of thalamus"
STR_MERFISH_Allen = MERFISH_Allen[MERFISH_Allen["ISH_STR"]==STR]
STR_MERFISH_MIT = MERFISH_MIT[MERFISH_MIT["ISH_STR"]==STR]
print("STR:{} Allen:{} MIT:{}".format(STR, STR_MERFISH_Allen.shape[0], STR_MERFISH_MIT.shape[0]))

In [ ]:
set(STR_MERFISH_Allen["class"])

# Figure out expl cal diff

In [ ]:
MERFISH = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/MERFISH.ISH_Annot.clean.csv", index_col=0)
Cluster_V3_Exp = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/SC_UMI_Mats/cluster_MeanLogUMI.csv", index_col=0)

In [ ]:
Non_Neuron_Class = ["30 Astro-Epen","31 OPC-Oligo","32 OEC","33 Vascular","34 Immune"]

In [ ]:
STR = "Superior vestibular nucleus"
MERFISH_STR = MERFISH[MERFISH["ISH_STR"]==STR]
MERFISH_STR_Neur = MERFISH_STR[~MERFISH_STR["class"].isin(Non_Neuron_Class)]

In [ ]:
test = MERFISH_STR[MERFISH_STR["cluster"]=="3464 SNr Six3 Gaba_1"]

In [ ]:
test.shape

In [ ]:
cluster_counts = MERFISH_STR_Neur['cluster'].value_counts()
cluster_counts_df = cluster_counts.reset_index()
cluster_counts_df.columns = ['Cluster', 'Count']

In [ ]:
gene = 481

In [ ]:
for i, row in cluster_counts_df.iterrows():
    cluster_counts_df.loc[i, "exp"] = Cluster_V3_Exp.loc[gene, row["Cluster"]]

In [ ]:
cluster_counts_df

In [ ]:
weighted_exp = (cluster_counts_df['exp'] * cluster_counts_df['Count']).sum() / cluster_counts_df['Count'].sum()

In [ ]:
(cluster_counts_df['exp'] * cluster_counts_df['Count']).sum()

In [ ]:
weighted_exp

In [ ]:
cluster_counts_df['Count'].sum()

In [ ]:
JC_cluster_counts = pd.read_csv("dat/JonDat/Debug_STR_CellComp.csv", index_col=0)
JC_cluster_counts = JC_cluster_counts.sort_values("n", ascending=False)
JC_cluster_counts = JC_cluster_counts[JC_cluster_counts["exp"]!="NaN"]

In [ ]:
JC_cluster_counts

In [ ]:
gene_idx = np.where(Cluster_V3_Exp.index.values==gene)[0][0]
gene_idx

In [ ]:
MERFISH_STR = MERFISH[MERFISH["ISH_STR"]==STR]

print("Processing {}; No.Cells:{}".format(STR, MERFISH_STR.shape[0]))
str_cell_sum_exp = np.zeros(Cluster_V3_Exp.shape[0])
str_neuro_sum_exp = np.zeros(Cluster_V3_Exp.shape[0])
str_neuro_count = 0 
neuro_type_dict = {}
neuro_type_gene_total_exp = 0
for i, row in MERFISH_STR.iterrows():
    cluster = row["cluster"]
    cell_class = row["class"]
    if cluster in Cluster_V3_Exp.columns.values:
        cell_exp = Cluster_V3_Exp[cluster].values
        #cell_exp = np.nan_to_num(cell_exp, nan=0)
        str_cell_sum_exp += cell_exp
        if cell_class not in Non_Neuron_Class:
            str_neuro_sum_exp += cell_exp
            str_neuro_count += 1
            neuro_type_gene_total_exp += cell_exp[gene_idx]
            if cluster not in neuro_type_dict:
                neuro_type_dict[cluster] = [1, cell_exp[gene_idx]]
            else:
                neuro_type_dict[cluster][0] += 1
if MERFISH_STR.shape[0] == 0:
    cell_avg_exp = str_cell_sum_exp
    neuro_avg_exp = str_neuro_sum_exp
else:
    cell_avg_exp = str_cell_sum_exp / MERFISH_STR.shape[0]
    neuro_avg_exp = str_neuro_sum_exp / str_neuro_count

In [ ]:
print(MERFISH_STR.shape[0], str_neuro_count)

In [ ]:
neuro_type_gene_total_exp

In [ ]:
neuro_type_dict

In [ ]:
exp_sum = 0
for k, v in neuro_type_dict.items():
    v1, v2 = v
    exp_sum += v1 * v2

In [ ]:
exp_sum / str_neuro_count

In [ ]:
str_neuro_sum_exp

In [ ]:
str_neuro_count

In [ ]:
neuro_avg_exp

In [ ]:
str_neuro_sum_exp[gene_idx]

In [ ]:
neuro_avg_exp_df = pd.DataFrame(data={"Gene":Cluster_V3_Exp.index.values, "exp":neuro_avg_exp, 
                                      "Neuro.sum.exp":str_neuro_sum_exp})
neuro_avg_exp_df = neuro_avg_exp_df.set_index("Gene")

In [ ]:
gene

In [ ]:
neuro_avg_exp_df.head(2)

In [ ]:
neuro_avg_exp_df.loc[481, :]

In [ ]:
np.where(str_neuro_sum_exp==681.655797)

In [ ]:
for i, row in neuro_avg_exp_df.iterrows():
    total_exp = row["Neuro.sum.exp"]
    if total_exp > 2440 and total_exp < 2440.1:
        print(i, row)

In [ ]:
df = pd.read_csv("dat/MERFISH/STR_NEU_Mean_DF.UMI.csv", index_col=0)
df2 = pd.read_csv("dat/MERFISH/STR_Cell_Mean_DF.UMI.csv", index_col=0)
df3 = pd.read_csv("dat/MERFISH/STR_Vol_Mean_DF.UMI.csv", index_col=0)

In [ ]:
df.loc[gene, STR]

In [ ]:
df2.loc[gene, STR]

In [ ]:
df3.loc[gene, STR]